# साधा इमेज वर्गीकरणकर्ता

या नोटबुकमध्ये तुम्हाला पूर्व-प्रशिक्षित न्यूरल नेटवर्क वापरून प्रतिमा वर्गीकृत कशा करायच्या ते शिकवले जाईल.

**तुम्ही काय शिकाल:**
- पूर्व-प्रशिक्षित मॉडेल कसे लोड करायचे आणि वापरायचे
- प्रतिमेची पूर्वतयारी
- प्रतिमांवर अंदाज कसे करायचे
- आत्मविश्वास गुण समजून घेणे

**वापर प्रकरण:** प्रतिमांमधील वस्तू ओळखणे (जसे "मांजर", "कुत्रा", "गाडी" इत्यादी) 

---


## चरण 1: आवश्यक लायब्ररी आयात करा

आपल्याला आवश्यक असलेल्या साधनांचा आयात करूया. जर तुम्हाला यापैकी सर्वकाही समजले नाही तरी चिंता करू नका!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## चरण 2: पूर्व-प्रशिक्षित मॉडेल लोड करा

आम्ही **MobileNetV2** वापरणार आहोत, जो आधीच लाखो प्रतिमांवर प्रशिक्षित केलेला न्यूरल नेटवर्क आहे.

याला **Transfer Learning** म्हणतात - म्हणजेच, दुसऱ्या कोणीतरी प्रशिक्षित केलेले मॉडेल वापरणे!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## चरण 3: सहाय्यक फंक्शन्स

आपल्या मॉडेलसाठी प्रतिमा लोड आणि तयार करण्यासाठी फंक्शन्स तयार करूया.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## चरण 4: नमुना प्रतिमांवर चाचणी करा

चला इंटरनेटवरील काही प्रतिमा वर्गीकृत करून पाहूया!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### प्रत्येक प्रतिमा वर्गीकृत करा


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## चरण ५: तुमच्या स्वतःच्या प्रतिमा वापरून पहा!

खाली दिलेल्या URL च्या जागी तुम्हाला वर्गीकृत करायची असलेली कोणतीही प्रतिमेची URL बदला.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 काय घडलं?

1. **आम्ही पूर्व-प्रशिक्षित मॉडेल लोड केले** - MobileNetV2 लाखो प्रतिमांवर प्रशिक्षित केले गेले आहे
2. **आम्ही प्रतिमा पूर्व-प्रक्रिया केल्या** - मॉडेलसाठी त्यांना आकार बदलून योग्य स्वरूपात तयार केले
3. **मॉडेलने अंदाज वर्तवले** - 1000 वस्तू वर्गांसाठी संभाव्यता दिल्या
4. **आम्ही निकाल डिकोड केले** - संख्यांना मानवी-वाचनीय लेबल्समध्ये रूपांतरित केले

### आत्मविश्वास गुण समजून घेणे

- **90-100%**: खूप आत्मविश्वासपूर्ण (जवळजवळ निश्चितपणे बरोबर)
- **70-90%**: आत्मविश्वासपूर्ण (बहुधा बरोबर)
- **50-70%**: थोडासा आत्मविश्वासपूर्ण (कदाचित बरोबर)
- **50% खाली**: फारसा आत्मविश्वासपूर्ण नाही (अनिश्चित)

### अंदाज चुकीचे का असू शकतात?

- **असामान्य कोन किंवा प्रकाश** - मॉडेलने सामान्य फोटोवर प्रशिक्षण घेतले आहे
- **अनेक वस्तू** - मॉडेल एक मुख्य वस्तू अपेक्षित करते
- **दुर्मिळ वस्तू** - मॉडेलला फक्त 1000 श्रेणी माहित आहेत
- **कमी दर्जाची प्रतिमा** - अस्पष्ट किंवा पिक्सेलयुक्त प्रतिमा ओळखणे कठीण आहे

---


## 🚀 पुढील पायऱ्या

1. **वेगवेगळ्या प्रतिमा वापरून पहा:**
   - [Unsplash](https://unsplash.com) वर प्रतिमा शोधा
   - उजव्या क्लिक करा → "Copy image address" करून URL मिळवा

2. **प्रयोग करा:**
   - अमूर्त कला वापरल्यास काय होते?
   - वेगवेगळ्या कोनातून वस्तू ओळखता येतात का?
   - अनेक वस्तूंसह ते कसे कार्य करते?

3. **अधिक जाणून घ्या:**
   - [Computer Vision lessons](../lessons/4-ComputerVision/README.md) एक्सप्लोर करा
   - स्वतःचा इमेज क्लासिफायर ट्रेन करायला शिका
   - CNNs (Convolutional Neural Networks) कसे कार्य करतात ते समजून घ्या

---

## 🎉 अभिनंदन!

तुम्ही अगदी अत्याधुनिक न्यूरल नेटवर्क वापरून एक इमेज क्लासिफायर तयार केला आहे!

हीच तंत्रज्ञान वापरले जाते:
- Google Photos (तुमच्या फोटोंचं आयोजन)
- स्वयंचलित कार (वस्तू ओळखणे)
- वैद्यकीय निदान (X-rays विश्लेषण)
- गुणवत्ता नियंत्रण (दोष शोधणे)

शोधत राहा आणि शिकत राहा! 🚀



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
